## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-08-01-15-45 +0000,nyt,Canada Culls Hundreds of Ostriches as a Court ...,https://www.nytimes.com/2025/11/07/world/canad...
1,2025-11-08-01-15-30 +0000,nyt,Trump Administration Appeals Judge’s Order to ...,https://www.nytimes.com/2025/11/07/us/politics...
2,2025-11-08-01-07-21 +0000,nyt,Routine Check Before UPS Plane Crash in Louisv...,https://www.nytimes.com/2025/11/07/us/ups-plan...
3,2025-11-08-01-07-00 +0000,wsj,Opinion | Elise Stefanik Takes on the Albany Left,https://www.wsj.com/opinion/elise-stefanik-new...
4,2025-11-08-01-06-38 +0000,nyt,Prince Harry Apologizes to Canadians for Weari...,https://www.nytimes.com/2025/11/07/world/canad...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2332/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,43
41,shutdown,14
47,up,13
170,flight,13
920,china,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
56,2025-11-07-22-23-00 +0000,wsj,"Republican Rep. Elise Stefanik, an ally of Pre...",https://www.wsj.com/politics/elections/elise-s...,114
247,2025-11-07-12-40-35 +0000,cbc,Trump administration fights court ruling to pa...,https://www.cbc.ca/news/world/us-ri-court-ruli...,102
348,2025-11-07-02-00-00 +0000,wsj,Allies of President Trump and Health Secretary...,https://www.wsj.com/politics/elections/trump-r...,90
327,2025-11-07-04-20-30 +0000,nypost,First lady Melania Trump receives ‘Patriot of ...,https://nypost.com/2025/11/06/us-news/first-la...,79
1,2025-11-08-01-15-30 +0000,nyt,Trump Administration Appeals Judge’s Order to ...,https://www.nytimes.com/2025/11/07/us/politics...,79


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
56,114,2025-11-07-22-23-00 +0000,wsj,"Republican Rep. Elise Stefanik, an ally of Pre...",https://www.wsj.com/politics/elections/elise-s...
294,76,2025-11-07-10-00-00 +0000,wsj,Democrats sought information about Social Secu...,https://www.wsj.com/politics/policy/senate-dem...
266,75,2025-11-07-11-00-00 +0000,latimes,Your flight was canceled amid FAA's cuts to ai...,https://www.latimes.com/california/story/2025-...
228,62,2025-11-07-14-11-00 +0000,wsj,Senate Majority Leader John Thune expects Sena...,https://www.wsj.com/politics/policy/government...
18,52,2025-11-07-23-56-29 +0000,nypost,Tesla rival Rivian gives CEO RJ Scaringe a Mus...,https://nypost.com/2025/11/07/business/rivian-...
337,45,2025-11-07-02-31-35 +0000,nypost,James Carville says he’d bet money Dems will w...,https://nypost.com/2025/11/06/us-news/james-ca...
196,40,2025-11-07-16-04-00 +0000,wsj,Heritage Foundation president Kevin Roberts de...,https://www.wsj.com/politics/policy/the-crack-...
14,38,2025-11-08-00-06-04 +0000,nypost,Michigan woman captured hurling piping hot cof...,https://nypost.com/2025/11/07/us-news/michigan...
344,34,2025-11-07-02-06-03 +0000,nypost,Erika Kirk receives inaugural ‘Charlie Kirk Le...,https://nypost.com/2025/11/06/us-news/erika-ki...
39,34,2025-11-07-22-58-07 +0000,nypost,Massive Bronx-to-Long Island drug pipeline bus...,https://nypost.com/2025/11/07/us-news/massive-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
